In [2]:
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 1. Loading the JSON 

In [3]:
fd = open('/kaggle/input/swiggy-restaurants-dataset/data.json','r')
js = fd.read()
fd.close()

js = json.loads(js)

### 2. Dataset Walkthrough - Abohar
##### 2.1) How many cities of data we're having in the dataset?

In [4]:
len(js.keys())

##### 2.2) How many restaurants are there in Abohar?

In [5]:
len(js['Abohar']['restaurants'].keys())

##### 2.3) All restaurant is serving how many number of different products category in Abohar?

In [6]:
for i in js['Abohar']['restaurants'].keys():
    print(js['Abohar']['restaurants'][i]['name'] ,'|', len(js['Abohar']['restaurants'][i]['menu']))

##### 2.4) Find the Restaurants with no Menu in Abohar?

In [7]:
for i in js['Abohar']['restaurants'].keys():
    if len(js['Abohar']['restaurants'][i]['menu']) == 0:
        print(js['Abohar']['restaurants'][i]['name'],'|', i)

#### 2.5) Different Crisines served in Abohar

In [8]:
cuisines = []

for i in js['Abohar']['restaurants'].keys():
    cuisines += js['Abohar']['restaurants'][i]['cuisine'].split(',')

cuisines = list(set(cuisines))
print(len(cuisines))

#### 2.5) Most Populat Cuisine served in Abohar (Top 5)

In [9]:
temp = []

for cuisine in cuisines:
    c = 0
    for i in js['Abohar']['restaurants'].keys():
        if (cuisine in js['Abohar']['restaurants'][i]['cuisine']):
            c += 1          
    temp.append([cuisine, c])
    
df_ = pd.DataFrame(temp, columns = ['cuisine','freq'])
df_.sort_values(by = 'freq', ascending = False).head()['cuisine']

#### 2.6) Average Cost of eating outside in Abohar

In [10]:
price = []
for i in js['Abohar']['restaurants'].keys():
    price.append(int(js['Abohar']['restaurants'][i]['cost'].split(' ')[-1]))
    
print(sum(price)//len(price))

#### 2.7) Average Cost of eating North Indian outside in Abohar 

In [11]:
price = []
for i in js['Abohar']['restaurants'].keys():
    if 'North Indian' in js['Abohar']['restaurants'][i]['cuisine']:
        price.append(int(js['Abohar']['restaurants'][i]['cost'].split(' ')[-1]))
    
print(sum(price)//len(price))

#### 2.8) Average Cost of Each Cuisine in Abohar

In [12]:
for cuisine in cuisines:
    price = []
    for i in js['Abohar']['restaurants'].keys():
        if cuisine in js['Abohar']['restaurants'][i]['cuisine']:
            price.append(int(js['Abohar']['restaurants'][i]['cost'].split(' ')[-1]))
    
    print(cuisine, sum(price)//len(price))

## 3. Dataset Walkthrough - Whole

##### 3.1) How many restaurents are there in each city?

In [24]:
temp = []

for i in js.keys(): 
    c = 0
    if ('restaurants' in js[i].keys()):
        c = len(js[i]['restaurants'])
        
    else:
        for region in js[i].keys():
            if (region != 'link'):
                c += len(js[i][region]['restaurants'])
    temp.append([i, c])

df_ = pd.DataFrame(temp, columns = ['city','restaurants'])
df_

##### 3.2) Top 5 Cities with Maximum number of Resturants

In [27]:
df_.sort_values(by = 'restaurants', ascending = False).head()

##### 3.3) How many restaurents have no menu or empty menu? (Null Data)

In [52]:
arr = []

c = 0

for city in js.keys():                                                # Going through each City
    
    if ('restaurants' in js[city].keys()):                            # If city is having no Subregions
        for rest in js[city]['restaurants'].keys():                   # Going through each restaurant
            if 'menu' in js[city]['restaurants'][rest].keys():        # Check if Menu is Present
                if len(js[city]['restaurants'][rest]['menu']) == 0:   # If Menu is Empty
                    c += 1
                    arr.append([rest])
            else:                                                     # If Menu is not Present
                c += 1
                arr.append([rest])
    else:                                                                               # If city is having Subregions
        for region in js[city].keys():                                                  # Going through each subregion
            if (region != 'link'):                                                      # Neglecting sub-region link
                for rest in js[city][region]['restaurants'].keys():                     # Finding all the restaurents
                    if 'menu' in js[city][region]['restaurants'][rest].keys():          # Check if Menu is Present
                        if len(js[city][region]['restaurants'][rest]['menu']) == 0:     # If Menu is Empty
                            c += 1
                            arr.append([rest])
                    else:                                                               # If Menu is not Present
                        c += 1
                        arr.append([rest])
print(c)

arr = np.array(arr)
np.save('Incompleted_Rest_data.npy',arr)

#### 3.4) Top 5 most expensive cities to eat outside?

In [85]:
df_ = []
c = 0

for city in js.keys():                                               
    cost = []
    if ('restaurants' in js[city].keys()):                           
        for rest in js[city]['restaurants'].keys():                  # Going through each restaurant
            try:
                cost.append(int(js[city]['restaurants'][rest]['cost'].split(' ')[-1]))
            except:
                pass

    else:                                                                             
        for region in js[city].keys():                                                
            if (region != 'link'):                                                    
                for rest in js[city][region]['restaurants'].keys():                     
                    try:
                        cost.append(int(js[city][region]['restaurants'][rest]['cost'].split(' ')[-1]))
                    except:
                        pass  
    try:
        df_.append([city , sum(cost)//len(cost)])
    except:
        pass
    
df_ = pd.DataFrame(df_, columns = ['city','cost'])
df_.sort_values(by = 'cost', ascending = False)

#### 3.5) Top 5 most expensive cities to eat Indian?

In [140]:
df_ = []
c = 0

for city in js.keys():                                               
    cost = []
    
    if ('restaurants' in js[city].keys()):                           
        for rest in js[city]['restaurants'].keys():                  # Going through each restaurant
            try:
                if ('Punjabi' in js[city]['restaurants'][rest]['cuisine']):
                    try:
                        cost.append(int(js[city]['restaurants'][rest]['cost'].split(' ')[-1]))
                    except:
                        pass
            except:
                pass
            
            try:
                cost = sum(cost)//len(cost)
            except:
                pass
    df_.append([city, cost])

df_ = pd.DataFrame(df_, columns = ['city','cost'])

cost = []
for i in df_['cost']:
    if(str(i) == '[]'):
        cost.append(0)
    else:
        cost.append(int(i))
        
df_['cost'] = cost
        
df_.sort_values(by = 'cost', ascending = False).head()